다른 사람들이 했던 거를 기반으로 Tracking back해서 무엇인갈 발견하는 것인가?  

https://lsjsj92.tistory.com/558  
https://lsjsj92.tistory.com/559?category=853217  

In [3]:
import os
import numpy as np
import pandas as pd
from glob import glob

In [2]:
os.listdir()

['submission38.csv',
 'Untitled.ipynb',
 'sub_TF_keras.csv',
 '.ipynb_checkpoints',
 'sub_blend009.csv',
 'sub_fcn.csv',
 'sub_200_ens_densenet.csv',
 'sub_keras_beginner.csv',
 'submission43.csv']

In [12]:
# sub_path = ''
all_files = glob('*.csv')
all_files[0]

'submission38.csv'

In [13]:
all_files = [all_files[1], all_files[3], all_files[0], all_files[6]]
all_files

['sub_TF_keras.csv', 'sub_fcn.csv', 'submission38.csv', 'submission43.csv']

In [14]:
# Read and concatenate submissions
out1 = pd.read_csv("sub_200_ens_densenet.csv", index_col=0)
out2 = pd.read_csv("sub_TF_keras.csv", index_col=0)
out3 = pd.read_csv("submission38.csv", index_col=0)
out4 = pd.read_csv("submission43.csv", index_col=0)
concat_sub = pd.concat([out1, out2, out3, out4], axis=1)
cols = list(map(lambda x: "is_iceberg_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3
0,5941774d,1.943216e-02,0.005586,0.008875,0.005586
1,4023181e,3.168809e-02,0.145927,0.146779,0.031688
2,b20200e4,4.000000e-08,0.000015,0.048646,0.001000
3,e7f018bb,9.925741e-01,0.999914,0.999184,0.999000
4,4371c8c3,2.215107e-02,0.033843,0.277251,0.022151


In [15]:
concat_sub.corr()

,is_iceberg_0,is_iceberg_1,is_iceberg_2,is_iceberg_3
is_iceberg_0,1.000000,0.890434,0.986291,0.994880
is_iceberg_1,0.890434,1.000000,0.923848,0.900871
is_iceberg_2,0.986291,0.923848,1.000000,0.988002
is_iceberg_3,0.994880,0.900871,0.988002,1.000000


In [16]:
# get the data fields ready for stacking
concat_sub['is_iceberg_max'] = concat_sub.iloc[:, 1:6].max(axis=1)
concat_sub['is_iceberg_min'] = concat_sub.iloc[:, 1:6].min(axis=1)
concat_sub['is_iceberg_mean'] = concat_sub.iloc[:, 1:6].mean(axis=1)
concat_sub['is_iceberg_median'] = concat_sub.iloc[:, 1:6].median(axis=1)

In [18]:
# set up cutoff threshold for lower and upper bounds, easy to twist
cutoff_lo = 0.8
cutoff_hi = 0.2

먼저 LB Score??
![image](https://user-images.githubusercontent.com/48716219/91177422-a9154500-e71e-11ea-8431-860078b64456.png)
캐글에서 제출한 score를 말하는 것 같다.

## Mean Stacking

In [20]:
concat_sub['is_iceberg'] = concat_sub['is_iceberg_mean']
concat_sub[['id', 'is_iceberg']].to_csv('stack_mean.csv', index=False, float_format='%.6f')

score가 0.1698점이 나왔다고 한다
- 나쁘지 않은 점수, 하지만 비교해보면 상위권의 모델들과는 차이가 있음

## Median Stacking

In [21]:
concat_sub['is_iceberg'] = concat_sub['is_iceberg_median']
concat_sub[['id', 'is_iceberg']].to_csv('stack_median.csv', index=False, float_format='%.6f')

0.1575점 => 상위 모델과 비슷한 퍼포먼스가 나왔다고 적혀있다. 좀더 좋은 SCORE를 얻어야 한다고 한다  

## PushOut + Median Stacking
- pushout이 뭔가
- np.all()?
    ![image](https://user-images.githubusercontent.com/48716219/91184024-19c05f80-e727-11ea-8a4e-b30a2eb3cc70.png)


In [22]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 1, 
                                   np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                            0, concat_sub['is_iceberg_median']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_pushout_median.csv', 
                                       index=False, float_format='%.6f')

0.1940 점수가 별로란다.

## MinMax + Mean Stacking
- minmax scaling?
- 이전보다 더 좋은 성능

In [23]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                   concat_sub['is_iceberg_max'], 
                                   np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                            concat_sub['is_iceberg_min'], 
                                            concat_sub['is_iceberg_mean']))
concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_mean.csv', 
                                       index=False, float_format='%.6f')

0.1622 => median으로 stacking 시도

## MinMax + Median Stacking

In [24]:
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                   concat_sub['is_iceberg_max'], 
                                   np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                            concat_sub['is_iceberg_min'], 
                                            concat_sub['is_iceberg_median']))
concat_sub['is_iceberg'] = np.clip(concat_sub['is_iceberg'].values, 0.001, 0.999)
concat_sub[['id', 'is_iceberg']].to_csv('stack_minmax_median.csv', 
                                      index=False, float_format='%.6f')

0.1488 => 작성자가 좋아한다. 상위 모델의 점수보다 좋아졌다고 한다.  
더 나아질 방법을 찾는다

## MinMax + BestBase Stacking

In [25]:
# load the model with best base performance
sub_base = pd.read_csv('submission43.csv')

In [26]:
concat_sub['is_iceberg_base'] = sub_base['is_iceberg']
concat_sub['is_iceberg'] = np.where(np.all(concat_sub.iloc[:,1:4] > cutoff_lo, axis=1), 
                                    concat_sub['is_iceberg_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:4] < cutoff_hi, axis=1),
                                             concat_sub['is_iceberg_min'], 
                                             concat_sub['is_iceberg_base']))
concat_sub['is_iceberg'] = np.clip(concat_sub['is_iceberg'].values, 0.001, 0.999)
concat_sub[['id', 'is_iceberg']].to_csv('submission54.csv', 
                                        index=False, float_format='%.6f')

Roboust model is always the key component, stacking only comes last with the promise to surprise, sometimes, in an unpleasant direction@.

Ruboust => robust???
굳건한 모델은 언제나 중요한 요소이다, stacking이 뭐 놀라운 결과를 가지고 온다.. 때때로 unpleasant한 방향으로